In [58]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

In [59]:
goal = pd.read_csv("./ronaldo_goal_prediction.csv")

In [60]:
goal

,Opponent Strength,Match Location,Ronaldo’s Form,Team Support,Scored
0,Strong,Away,Good,High,No
1,Weak,Home,Excellent,High,Yes
2,Medium,Away,Average,Low,No
3,Weak,Home,Good,Medium,Yes
4,Strong,Home,Excellent,High,Yes
5,Medium,Away,Poor,Low,No
6,Weak,Away,Good,High,Yes
7,Medium,Home,Average,Medium,No
8,Strong,Home,Excellent,Medium,Yes
9,Weak,Away,Poor,High,No


In [61]:
goal.describe()

,Opponent Strength,Match Location,Ronaldo’s Form,Team Support,Scored
count,14,14,14,14,14
unique,3,2,4,3,2
top,Weak,Home,Good,High,Yes
freq,5,8,4,7,8


In [64]:
class TreeNode:
  def __init__(self, value):
    self.value = value
    self.left = None
    self.right = None

In [138]:
def Gini(source, x, y):
  data = source[[x, y]]
  #print(data)
  
  unique_feature_vals = data[x].unique()
  unique_target_vals = data[y].unique()
  
  stats = pd.DataFrame(index=unique_feature_vals, columns=[*unique_target_vals, "total"])
  
  length = len(data)
  
  for value in unique_feature_vals:
    total = 0
    for target_value in unique_target_vals:
      count = len(data[(data[x] == value) & (data[y] == target_value)])
      stats.loc[value, target_value] = count
      total += count
    stats.loc[value, "total"] = total
  
  #print(stats)
  
  gini_cat = []
  
  for value in unique_feature_vals:
    p = stats.loc[value, unique_target_vals] / stats.loc[value, "total"]
    gini_cat.append(1 - sum(p**2))
  
  p_cat = stats["total"] / length
  
  return sum(np.array(gini_cat) * p_cat)  

In [145]:
X = goal.drop(columns=["Scored"])
y = goal["Scored"]

gini_vals = []

for column in X.columns:
    g = Gini(goal, column, "Scored")
    gini_vals.append(g)

gini = pd.DataFrame({"gini": gini_vals}, index=X.columns)

gini.loc[gini.idxmin()]

,gini
Ronaldo’s Form,0.214286
